This notebook was used to construct the examples in `07embedding/embedding_worksheet.docx`.

In [30]:
import torch
from torch import tensor
import pandas as pd

In [31]:
torch.manual_seed(0)
n_dim = 3
U = torch.randn(5, n_dim).round()
M = torch.randn(3, n_dim).round()


In [32]:
pd.DataFrame(M.numpy()).to_clipboard(header=False)

In [33]:
pd.Series([f'User {i+1}' for i in range(5)]).to_clipboard(index=False, header=False)

In [34]:
M

tensor([[ 0., -1.,  0.],
        [-1., -2.,  0.],
        [ 1.,  1.,  1.]])

In [35]:
pairs = [(1, 0), (0, 1), (3, 2)]
for uu, mm in pairs:
    print(f"dot(user {uu+1}, movie {mm+1}) = {U[uu] @ M[mm]}")

dot(user 2, movie 1) = 1.0
dot(user 1, movie 2) = -2.0
dot(user 4, movie 3) = -1.0


Constructing embeddings

Targets:

In [36]:
uu = torch.tensor([
    [1.0, -0.5],
    [0.0, 1.0],
])
vv = torch.tensor([
    [1.0, 1.0],
    [0.0, 1.0],
])
uu @ vv.T


tensor([[ 0.5000, -0.5000],
        [ 1.0000,  1.0000]])

In [40]:
ratings = torch.tensor([
  [1.0, 0.0, -1.0],
  [0.0, -1.0, 1.0]
])

In [38]:
# compute the PCA of ratings
# subtract the mean from each row
ratings_centered = ratings - ratings.mean(dim=-1, keepdim=True)
ratings_centered


tensor([[ 1.,  0., -1.],
        [ 0., -1.,  1.]])

In [39]:
# compute the SVD
svd_result = torch.svd(ratings_centered)
svd_result

torch.return_types.svd(
U=tensor([[ 0.7071,  0.7071],
        [-0.7071,  0.7071]]),
S=tensor([1.7321, 1.0000]),
V=tensor([[ 4.0825e-01,  7.0711e-01],
        [ 4.0825e-01, -7.0711e-01],
        [-8.1650e-01,  3.4572e-08]]))

In [54]:
U = torch.tensor([[1.0], [-1.0]])
V = torch.zeros(3, 1, requires_grad=True)

In [55]:
U[0] @ V[0]

tensor(0., grad_fn=<DotBackward0>)

In [56]:
loss = (U[0] @ V[0] - 1.0) ** 2
loss

tensor(1., grad_fn=<PowBackward0>)

In [57]:
loss.backward()
V.grad

tensor([[-2.],
        [ 0.],
        [ 0.]])

In [58]:
V.data -= 0.1 * V.grad

In [59]:
loss = (U[0] @ V[0] - 1.0) ** 2
loss

tensor(0.6400, grad_fn=<PowBackward0>)